Meredith Synnott

In [0]:
import requests
from time import sleep
import pandas as pd
import numpy as np
import warnings

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#load data and preview 
DATA_URL = 'https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv'
df = pd.read_csv(DATA_URL)

In [0]:
#create lists to store values from census 
allcbsas = []
allnames = []
incomedemocount = []
hispanicdemocount = []
agedemocount = []

#tables 
tableids = ['B19001', 'B01001', 'B03002']
tableidstring = ','.join(tableids)
tableids

#parameter 1 HHincome 50k-74.9k
#parameter 2 18-34
#parameter 3 large hispanic populations 

def getdata(url):
    data = requests.get(url).json()
    return(data)
requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03002&geo_ids=310|01000US' 

loadedjson = requests.get(requesturl)


#for loop
count = 0
for msa in df:
    count += 1
    #skipping header row of csv file
    if count != 1:

        #pull out cbsa, append to list
        cbsa = msa[0]
        
        allcbsas.append(cbsa)
        #pull out name of cbsa, append to list
        name = msa[1]
        allnames.append(name)
        
        #create url for a specific msa
        url = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US%s' % (cbsa)
        print(url)
        #get data for cbsa
        data = getdata(url)


https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000USN
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000USP


In [0]:
   # income (#1)
        # parse through the dictionary returned and get just the value we need
        inc = data['data']['31000US%s' % (cbsa)]['B19001']['estimate']
        inc50 = inc['B19001011']
        inc60 = inc['B19001012']
        totalincomecount = inc50 + inc60
#write that value to the corresponding list of values
        incomedemocount.append(totalincomecount)
        
        # age (#2)
        # parse through the dictionary returned and get just the value we need
        age = data['data']['31000US%s' % (cbsa)]['B01001']['estimate']
        ageM18_19 = age['B01001007']
        ageM20 = age['B01001008']
        ageM21 = age['B01001009']
        ageM22_24 = age['B01001010']
        ageM25_29 = age['B01001011']
        ageM30_34 = age['B01001012']
        ageF18_19 = age['B01001031']
        ageF20 = age['B01001032']
        ageF21 = age['B01001033']
        ageF22_24 = age['B01001034']
        ageF25_29 = age['B01001035']
        ageF30_34 = age['B01001036']
        ageT = ageM18_19 + ageM20 + ageM21 + ageM22_24 + ageM25_29 + ageM30_34 \
            + ageF18_19 + ageF20 + ageF21 + ageF22_24 + ageF25_29 + ageF30_34
        #write that value to the corresponding list of values
        agedemocount.append(ageT)

        # hispanic (#3)
        # parse through the dictionary returned and get just the value we need
        pop_hisp = data['data']['31000US%s' % (cbsa)]['B03003']['estimate']['B03003003']
        #write that value to the corresponding list of values
        hispanicdemocount.append(pop_hisp)

KeyError: ignored

In [0]:
#create a new dataframe
df = pd.DataFrame()

#write the lists to columns, one by one
df['cbsa'] = allcbsas
df['name'] = allnames
df['income'] = incomedemocount
df['hispanic'] = hispanicdemocount
df['age'] = agedemocount

ValueError: ignored

In [0]:
#find out the max values
#max income
print('Q1: max income')
print('-------------------------')
df.iloc[df.income.argmax()]

#max age
print('Q2: max age')
print('-------------------------')
df.iloc[df.age.argmax()]

#max hispanic
print('Q3: max hispanic')
print('-------------------------')
df.iloc[df.hispanic.argmax()]

df['meanincome'] = df['income']/df['income'].max()
df['meanhispanic'] = df['hispanic']/df['hispanic'].max()                
df['meanage'] = df['age']/df['age'].max()  

#mean of the 3 means 
df['totalmean'] = (df['meanincome'] + df['meanhispanic'] + df['meanage']) / 3

#highest mean wins
print('Q4: highest cumulative mean')
print('-------------------------')
df.iloc[df.totalmean.argmax()]

Q1: max income
-------------------------


AttributeError: ignored